In [1]:
import overpy
import shapefile
import urllib.request

def export_shp(obj_name, shape_type):
    #download osm data
    api = overpy.Overpass()
    result = api.query("""
        way({},{},{},{}) ["{}"];
        (._;>;);
        out body;
        """.format(bbox[1], bbox[0], bbox[3], bbox[2], obj_name))
    params = [[way.tags.get("name", "n/a"), way.tags.get(obj_name, "n/a")] for way in result.ways]
    coords = [[[float(node.lon), float(node.lat)] for node in way.nodes] for way in result.ways]
    '''
    for way in result.ways:
        print(way.tags)
    '''
    #export to shapefile
    w = shapefile.Writer(shapeType=shape_type)
    w.fields = [['NAME', 'C', 150, 0], ['TYPE', 'C', 50, 0]]
    w.records.extend(params)  #fill with values
    for line in coords:
        w.poly(shapeType=3, parts=[line])  #add geometry for previously filled values
    w.save(obj_name)
    prj = urllib.request.urlopen("http://spatialreference.org/ref/epsg/4326/esriwkt/").read()  #add a projection file
    with open(obj_name+'.prj', "w") as f:
        f.write(prj.decode('utf-8'))
        
inshp = "area/reg_buff.shp"
sf = shapefile.Reader(inshp)
test_tag = "boundary"
bbox = sf.bbox
res = export_shp(test_tag, shapefile.POLYLINE)

In [2]:
object_list = {shapefile.POLYLINE: ['building', 'highway', 'junction', 'railway', 'route', 'landuse']} #shapefile.POINT: ['amenity','landmark', ]